Link to github repo: https://github.com/Edyarich/dlsystems-final-project

### **1) Math part** 
To understand how to code this kind of model, first we need to explore math intuition behind. This paperы were used to understand all the background: \\
1) https://arxiv.org/abs/2105.05233 \\
2) https://arxiv.org/abs/2102.09672 \\
All in all we have such a **pipeline**: \\
Model has two passes - forward and backward. **Forward process** is following:

> We need to make the pure noise from our image, by adding some noise from a Gaussian distrubution several times. The mean and variance are parameters, which we need to generate some noise. Actually we will change only mean, so let us take the variance out of speech (**PROBABLY SHOULD CHANGE THIS SENTENCE**). The way how we change our mean, depending on the timestep is called a **schedule**. We have implemented two schedules: linear and cosine, but there are a lot more variants to generate means for our noises. Linear schedule just means that we have beggining and ending numbers for the mean value and we make a linear function with f(0) = beggining, f(T) = ending; where T - number of timesteps. \\
$q(x_t|x_{t-1}) := \mathcal{N} (x_t;\sqrt{1-β_t}x_{t-1}, β_t𝐈)$ \\
In code we will not apply this forward function *n* times to get *n_noised* image. There is a way, how we can recalculate the mean from schedule and jump from input image to the image at step *n*. More formulas and proves are presented in the following papers and the result are used in code, in __init__ function for Diffusion model class. \\
Denote $α_t := 1 - β_t$ and $α_{t}^{'} := ∏_{s=0}^t α_s$ \\
Then forward process will look like: \\
$q(x_t|x_0) = \mathcal{N} (x_t; \sqrt{α_{t}^{'}}x_0, (1-α_{t}^{'})𝐈)$ \\
Also forward process function is denoted by **q** and in code we follow this mark, so all q_sample function or etc. are about forward process

Now let us talk about the most interesting part - **Backward process**:
> In backward process (denoted by **p**, same logic with sentence above) we want to predict image before noising, when the given input is noised image. Since forward process is about adding the noise tensors to our input tensor, we can simplify the goal to predict the noise, which was added at *ith* step. Authors of DDPM model itself and authors of papers provided us with results and pipeline of the experience, which they made about how exactly we can predict the noise. Final decision and answer is just to predict the mean to generate the noise same as in forward process. \\
At this part we need to choose the deep learning model, which will be used to do this prediction. Our decision is to use UNet architecture. We built it from scratch, also building new operations and modules, but more about this in further proposal. \\
In conslusion, we will learn our model to predict the noise from input image and given timestep and backward pass will be done. Now, to generate new sample we will give our model some pure noise and it will be recovering the image from it, as practice shows, we will get some brand new images. \\
$p_θ(x_{t-1}|x_t) := \mathcal{N}(x_{t-1}; μ_{θ}(x_t, t), ∑_{θ}(x_t, t))$, where $θ$ we try to learn

### **2) Diffusion class** 
In **apps/diffusion.py** we started with implementing the Diffusion class. It is a main block of our new model. It also requires some helper functions: 
> **2.1) extract** makes the tensor, given the betas array and timesteps subarray \\
```python
  def extract(x, t, x_shape):
    batch_size = t.shape[0]
    device = x.device
    out_handle = device.empty((batch_size,))
    for i in range(batch_size):
        ind = int(t.numpy()[i])
        out_handle[i] = x.cached_data[ind]
    new_shape = (batch_size,) + (1,) * (len(x_shape) - 1)
    return Tensor(out_handle, device=device).reshape(new_shape)
```
**2.2) schedules** what is time schedule we discussed in math block, so there are implementations of two types of them: linear and cosine
```python
  def linear_beta_schedule(timesteps, device=None):
      scale = 1000 / timesteps
      beta_start = scale * 0.0001
      beta_end = scale * 0.02
      return Tensor(array_api.linspace(beta_start, beta_end, timesteps),
                    dtype="float32", device=device)
```
>
 At the initialization moment we setup all the preferences: 
 > **a) noise schedule** - the rule, how noise will be added at each forward step. there are two options: linear and cosine schedule \\
**b) denoising model** - model, which will be used to predict the noise and make the backward process work, more about this part a bit later \\
**c) timesteps** - just a number of noising steps \\
**d)** **loss** function and **device** where model exists. 
>
Our class methods are: \\

>**2.1) q_sample** function applies forward noising process. Uses extract [2.1] function and some pre-calculated data (more in math block) from betas, which were given by time schedule. \\
```python
  def q_sample(self, x_0, t, noise=None):
        shape = x_0.shape
        noise = x_0.device.randn(*shape) if noise is None else noise
        return (
            (extract(self.sqrt_alphas_cumprod, t, x_0.shape).broadcast_to(shape) * x_0 +
             extract(self.sqrt_one_minus_alphas_cumprod, t, x_0.shape).broadcast_to(shape) * noise).data
        )
```
**2.2) p_sample** - function to apply single step of denoising process. It uses given model to predict the noise on exact step. Also we do some betas (from schedule usage) transformations, such as making tensor from them, then generating cocoefficients to jump right into the ith step. \\
```python
  def p_sample(self, model, x, t, t_index):
        betas_t = extract(self.betas, t, x.shape).data.broadcast_to(x.shape)
        sqrt_one_minus_alphas_cumprod_t = extract(
            self.sqrt_one_minus_alphas_cumprod, t, x.shape
        ).data.broadcast_to(x.shape)
        sqrt_recip_alphas_t = extract(
            self.sqrt_recip_alphas, t, x.shape
        ).data.broadcast_to(x.shape)
        model_mean = (sqrt_recip_alphas_t * (
                x - betas_t * model(x, t) / sqrt_one_minus_alphas_cumprod_t
        )).data
        if t_index == 0:
            return model_mean
        else:
            posterior_variance_t = extract(
                self.posterior_variance, t, x.shape
            ).data.broadcast_to(x.shape)
            noise = init.randn(*x.shape, device=x.device, requires_grad=False)
            return model_mean + ops.sqrt(posterior_variance_t) * noise
```
**2.3) sample_loop** - function, which applies all the denoising steps, just returning the result of **p_sample_loop** with following logic: \\
**a)** generation of pure noise image with help of init.randn function \\
**b)** apply **p_sample** function to an previous image (image always changing, during applying this functions) with given model
**c)** return the array, where we have stored all the steps of denoising an image
 \\
```python
  def p_sample_loop(self, shape):
        model = self.denoise_model
        device = model.parameters()[0].device
        batch_size = shape[0]
        img = init.randn(*shape, device=device, requires_grad=False)
        imgs = []
        for i in tqdm(reversed(range(0, self.timesteps)),
                      desc='sampling loop time step', total=self.timesteps):
            img = self.p_sample(
                model,
                img,
                init.constant(
                    batch_size,
                    c=i,
                    device=device,
                    requires_grad=False
                ),
                t_index=i
            )
            imgs.append(img.detach().numpy())
        return imgs
```
>

###**3) UNetBlock class** 
Really massive work has been done on this predictioning model and UnetBlock is a core part of the Unet class, which will be described a bit later. First, let us talk about operations, which were added to `needle/ops.py`: **ConvTranspose** and **MaxPool** \\

Block contains such layers: 
> **a)** Time embedding with nn.Linear layer and then nn.ReLU in forward function \\
 **b)** Convolution layer with arguments, depend on which UNet part we use this block (up or down) \\
**c)** Second convolution \\
**d)** Batch normalization over the result of the successively used layers on the input \\
> **e)** Returned data is a transformation, applied to result of **d)** step. If we are upsampling - this transformation is ConvTranspose, if we downsampling it will be MaxPool
```python
  class UnetBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim, up=False, device=None):
        super().__init__()
        self.time_mlp = nn.Linear(time_emb_dim, out_ch, device=device)
        if up:
            self.conv1 = nn.Conv(2 * in_ch, out_ch, 3, padding=1, device=device)
            self.transform = nn.ConvTranspose(out_ch, out_ch, 4, 2, 1, device=device)
        else:
            self.conv1 = nn.Conv(in_ch, out_ch, 3, padding=1, device=device)
            self.transform = nn.MaxPool(2)
        self.conv2 = nn.Conv(out_ch, out_ch, 3, padding=1, device=device)
        self.bnorm1 = nn.BatchNorm2d(out_ch, device=device)
        self.bnorm2 = nn.BatchNorm2d(out_ch, device=device)
        self.relu = nn.ReLU()
    def forward(self, x, t):
        h = self.bnorm1(self.relu(self.conv1(x)))
        time_emb = self.relu(self.time_mlp(t))
        time_emb = time_emb.reshape(time_emb.shape + (1, 1)).broadcast_to(h.shape)
        h = h + time_emb
        h = self.bnorm2(self.relu(self.conv2(h)))
        return self.transform(h)
```

### **3.1) ConvTranspose**
As a matter of fact, ConvTranspose is a combination of two layers: Upsampling and Convolution.  
So, our implementation do exactly the same: it modifies the input using `ops.dilate` and then applies convolution.

```python
class ConvTranspose(Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int,
                 stride: int = 1, padding: int = 0, output_padding: int = 0,
                 bias: bool = True, device=None, dtype: str = "float32"):
        super().__init__()

        conv_padding = output_padding + kernel_size - 1 - padding
        self.conv = Conv(in_channels, out_channels, kernel_size, 1,
                         conv_padding, bias, device, dtype, True)
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        dilated_x = ops.dilate(
            x, axes=(2, 3), dilation=self.stride - 1, cut_last=True
        )
        return self.conv(dilated_x)
```

### **3.2) MaxPool**


We have implemented the basic version of maxpooling with stride = kernel_size, with no padding and dilation.
There are two reasons for this decision
1. No need for the modified MaxPool layer, the simple one is OK for UNet
2. The gradient calculation of Maxpool layer becomes more sophisticated when we take additional parameters into consideration

```python
class MaxPool(Module):
    def __init__(self, kernel_size):
        super().__init__()

        self.kernel_size = kernel_size

    def forward(self, x: Tensor) -> Tensor:
        # NCHW ==> NHWC ==> NH'W'O ==> NOH'W'
        _x = x.permute((0, 2, 3, 1))
        output = ops.maxpool(_x, self.kernel_size)
        return output.permute((0, 3, 1, 2))
```

### **3.3) State dictionary**
State dictionary is a simple dictionary which contains parameters of the model. They can be easily saved, updated and restored, adding a great deal of modularity to needle models  It is a useful tool for model checkpointing during the training process.  
To begin with, we implemented two magic methods `__getstate__` and `__setstate__`, which are necessary for correct pickle serialization.
Then, we modified `nn.Module`, adding methods for loading and saving state dictionaries.

```python
class NDArray:
    ...
    def __getstate__(self):
        # Method for correct pickle serialization of NDArray
        # https://codefather.tech/blog/python-pickle/
        return {'data': self.numpy(), 'device': self.device.name}

    def __setstate__(self, state):
        self.__init__(state['data'], _find_device(state['device']))
```

###**4) UNet class**
The class takes all required information about the shape of the images, channels count and time encoding. This preferences will be used in the layers and block initializations. All the timesteps first should be encoded with sinusoidal positional embedding, which is implemented in `SinusoidalPosEmb` class. Required functions of sinus and cosinus were added both in backend ndarray and in `needle/ops.py` \\
Then it builds the complete UNet architecture: \\
> **a)** Time embedding is actually block of connected layers by `nn.Sequential`: `SinPosEmb`, `nn.Linear` and `nn.ReLU` layers. \\
**b)** Projection is a simple convolution \\
**c)** Then we initialize two list for downsampling and upsampling branches and fill this lists with already implemented UNetBlocks with parameters from input arguments \\
**d)** In forward process the model predicts noise having noisy image and its timestamp
> 

###**5) Learning model**
The whole pipeline is available in `main.ipynb`  
**5.1) Dataset** We chose landscapes dataset from kaggle: https://www.kaggle.com/datasets/arnaud58/landscape-pictures?resource=download \\
Following **LandscapesDataset class** was added in **needle/data.py** and we instantly apply resize and normalize transformation for more accurate work of the model and faster learning. \\
**5.2) Learning.ipynb** First cells are required to setup cuda, build the cpp files, clone the code from github and download the landscapes dataset. To train we chose this parameters: optimizer: Adam, loss: L2, timesteps (number of forward and backward steps): 300, 2 epoches and batch consist of 12 images. \\
**Training process** took us 1 hour with a result l2 loss of 0.324



-------------------

# BACKUP

### Work by Eduard and Michael.
Our work on this project consist of several parts, all of them will be described in further text.

---


### **1) Math part** 
To understand how to code this kind of model, first we need to explore math intuition behind. This paperы were used to understand all the background: \\
1) https://arxiv.org/abs/2105.05233 \\
2) https://arxiv.org/abs/2102.09672 \\
All in all we have such a **pipeline**: \\
Model has two passes - forward and backward. **Forward process** is following:

> We need to make the pure noise from our image, by adding some noise from a Gaussian distrubution several times. The mean and variance are parameters, which we need to generate some noise. Actually we will change only mean, so let us take the variance out of speech (**PROBABLY SHOULD CHANGE THIS SENTENCE**). The way how we change our mean, depending on the timestep is called a **schedule**. We have implemented two schedules: linear and cosine, but there are a lot more variants to generate means for our noises. Linear schedule just means that we have beggining and ending numbers for the mean value and we make a linear function with f(0) = beggining, f(T) = ending; where T - number of timesteps. \\
In code we will not apply this forward function *n* times to get *n_noised* image. There is a way, how we can recalculate the mean from schedule and jump from input image to the image at step *n*. More formulas and proves are presented in the following papers and the result are used in code, in __init__ function for Diffusion model class. Also forward process function is denoted by **q** and in code we follow this mark, so all q_sample function or etc. are about forward process
>
Now let us talk about the most interesting part - **Backward process**:
> In backward process (denoted by **p**, same logic with sentence above) we want to predict image before noising, when the given input is noised image. Since forward process is about adding the noise tensors to our input tensor, we can simplify the goal to predict the noise, which was added at *ith* step. Authors of DDPM model itself and authors of papers provided us with results and pipeline of the experience, which they made about how exactly we can predict the noise. Final decision and answer is just to predict the mean to generate the noise same as in forward process. \\
At this part we need to choose the deep learning model, which will be used to do this prediction. Our decision is to use UNet architecture. We built it from scratch, also building new operations and modules, but more about this in further proposal. \\
In conslusion, we will learn our model to predict the noise from input image and given timestep and backward pass will be done. Now, to generate new sample we will give our model some pure noise and it will be recovering the image from it, as practice shows, we will get some brand new images.

---


### **2) Diffusion class** 
In **apps/diffusion.py** we started with implementing the Diffusion class. It is a main block of our new model. It also requires some helper functions: 
> **2.1) extract** TODO \\
**2.2) schedules** what is time schedule we discussed in math block, so there are implementations of two types of them: linear and cosine
>
 At the initialization moment we setup all the preferences: 
 > **a) noise schedule** - the rule, how noise will be added at each forward step. there are two options: linear and cosine schedule \\
**b) denoising model** - model, which will be used to predict the noise and make the backward process work, more about this part a bit later \\
**c) timesteps** - just a number of noising steps \\
**d)** **loss** function and **device** where model exists. 
>
Our class methods are: \\

>**2.1) q_sample** function applies forward noising process. Uses extract [2.1] function and some pre-calculated data (more in math block) from betas, which were given by time schedule. \\
**2.2) p_sample** - function to apply single step of denoising process. It uses given model to predict the noise on exact step. Also we do some betas (from schedule usage) transformations, such as making tensor from them, then generating cocoefficients to jump right into the ith step. \\
**2.3) sample_loop** - function, which applies all the denoising steps, just returning the result of **p_sample_loop** with following logic: \\
**a)** generation of pure noise image with help of init.randn function \\
**b)** apply **p_sample** function to an previous image (image always changing, during applying this functions) with given model
**c)** return the array, where we have stored all the steps of denoising an image
 \\
>
---
###**3) UNetBlock class** 
Really massive work has been done on this predictioning model and UnetBlock is part of all the Unet class, which will be described a bit later. First, let us talk about operations, which were added to **needle/ops.py** \\
**3.1) ConvTranspose** TODO \\
**3.2) MaxPool** TODO \\
Block contains such a layers: 
> **a)** Time embedding with nn.Linear layer and then nn.ReLU in forward function \\
 **b)** Convolution layer with arguments, depend on which UNet part we use this block (up or down) \\
**c)** Second convolution \\
**d)** Batch normalization over the result of the successively used layers on the input \\
> **e)** Returned data is transformation, applied to result of **d)** step. If we are upsampling - this transformation is ConvTranspose, if we downsampling it will be MaxPool

---



###**4) UNet class**
As input this class takes all required parameters about the images shape and channel numbers. Also takes the info, which will be needed in time encoding. This preferences will be used in the layers and block initializations. All the timesteps first should be encoded with sinusoidal positional embedding, which is implemented in **SinusoidalPosEmb** class. Required functions of sinus and cosinus were added both in backend ndarray and in needle/ops.py \\
Then it builds the complete UNet architecture: \\
> **a)** Time embedding is actually block of connected layers by nn.Sequential: SinPosEmb, nn.Linear and nn.ReLU layers. \\
**b)** Projection is a simple convolution \\
**c)** Then we initialize two list for downsampling and upsampling branches and fill this lists with already implemented UNetBlocks with parameters from input arguments \\
**d)** In forward process we **TODO**
> 

---
###**5) Learning model**
**5.1) Dataset** We chose landscapes dataset from kaggle: https://www.kaggle.com/datasets/arnaud58/landscape-pictures?resource=download \\
Following **LandscapesDataset class** was added in **needle/data.py** and we instantly apply resize and normalize transformation for more accurate work of the model and faster learning. \\
**5.2) Learning.ipynb** First cells are required to setup cuda, build the cpp files, clone the code from github and download the landscapes dataset. To train we chose this parameters: optimizer: Adam, loss: L2, timesteps (number of forward and backward steps): 300, 4 epoches and batch consist of 4 images. \\
**Training process** took us **TODO TIME** with a result loss of **TODO LOSS** 